In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,Dropout,AveragePooling2D,GlobalAveragePooling2D,Reshape
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import layers,Model
# import numba
import tensorflow as tf

In [ ]:
img_width,img_height = 224,224
train_data_dir = "dataset\Training_Data"
validation_data_dir = "dataset\Test_Data"
batch_size = 8
nb_validation_samples = 100

model  = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_width, img_height,3))

model.summary()


In [ ]:
x = model.output
#x = Flatten()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1536, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model_final = Model(input = model.input, output = output)

model_final.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")


In [ ]:
with tf.device("/gpu:1"):
    model_final.fit_generator(train_generator,samples_per_epoch = 1000,epochs = 30,validation_data = validation_generator,nb_val_samples = nb_validation_samples)


In [ ]:
model_final.save('inception_resnet_v2_model__30_meet.h5')